<a href="https://colab.research.google.com/github/natsakh/IAD/blob/main/Pr_7/7_4_Encoder_Decoder_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Translation

https://keras.io/examples/nlp/neural_machine_translation_with_transformer/

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
from collections import Counter
import torch.nn.functional as F

import random
import math

torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


#Tab-delimited Bilingual Sentence Pairs
https://www.manythings.org/anki/


In [2]:
!wget https://www.manythings.org/anki/spa-eng.zip
!unzip -o spa-eng.zip

--2025-11-21 17:54:08--  https://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5466500 (5.2M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   5.21M  3.12MB/s    in 1.7s    

2025-11-21 17:54:10 (3.12 MB/s) - ‘spa-eng.zip’ saved [5466500/5466500]

Archive:  spa-eng.zip
  inflating: _about.txt              
  inflating: spa.txt                 


In [3]:
lines = open("spa.txt", encoding="utf-8").read().split("\n")
print(lines[0])
print()

pairs = []
for line in lines:
    parts = line.split("\t")
    if len(parts) < 2:
        continue
    eng = parts[0].strip()
    spa = parts[1].strip()
    pairs.append((eng, spa))

print(pairs[0])
print(len(pairs))
print()

for _ in range(5):
    print(random.choice(pairs))

Go.	Ve.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #4986655 (cueyayotl)

('Go.', 'Ve.')
142928

('Do you have a violin?', '¿Tienes un violín?')
('This is my CD.', 'Es mi CD.')
('She made him a simple dinner.', 'Ella le hizo una cena sencilla.')
('Are you guys having any fun?', '¿Os estáis divirtiendo?')
('We swam until it got dark.', 'Nadamos hasta que oscureció.')


In [4]:
MAX_VOCAB = 15000
specials = ["[PAD]", "[UNK]", "[BOS]", "[EOS]"]


# Токенізатор (простий)
def tokenize(text):
    return text.lower().strip().split()

In [5]:
# Будуємо словник
def build_vocab(sentences):
    counter = Counter()

    for s in sentences:
        counter.update(tokenize(s))

    most_common = counter.most_common(MAX_VOCAB - len(specials))
    itos = specials + [w for w, _ in most_common]
    stoi = {w: i for i, w in enumerate(itos)}

    return stoi, itos


In [6]:
# Розділяємо дані
eng_sentences = [p[0] for p in pairs]
spa_sentences = [p[1] for p in pairs]

# Створюємо два словники
eng_stoi, eng_itos = build_vocab(eng_sentences)
spa_stoi, spa_itos = build_vocab(spa_sentences)

PAD_IDX = eng_stoi["[PAD]"]    # однакові індекси у двох мовах — добре
UNK_IDX = eng_stoi["[UNK]"]
BOS_IDX = eng_stoi["[BOS]"]
EOS_IDX = eng_stoi["[EOS]"]

print("ENG vocab size:", len(eng_stoi))
print("SPA vocab size:", len(spa_stoi))

ENG vocab size: 15000
SPA vocab size: 15000


In [7]:
MAX_LEN_SRC = 20   # довжина англ речення
MAX_LEN_TGT = 20   # довжина іспанського (включно з BOS/EOS)

def encode(text, stoi, add_specials=False):
    tokens = tokenize(text)
    ids = []

    if add_specials:
        ids.append(BOS_IDX)

    for t in tokens:
        ids.append(stoi.get(t, UNK_IDX))

    if add_specials:
        ids.append(EOS_IDX)

    return ids

In [8]:
def prepare_pair(eng, spa):
    # encoder input: англ без BOS/EOS
    src = encode(eng, eng_stoi, add_specials=False)

    # повна цільова послідовність: [BOS ... EOS]
    tgt_full = encode(spa, spa_stoi, add_specials=True)

    # decoder_inputs: без останнього токена
    tgt_in = tgt_full[:-1]

    # targets: без першого токена (зсунуті)
    tgt_out = tgt_full[1:]

    # паддинг
    src = src[:MAX_LEN_SRC] + [PAD_IDX] * (MAX_LEN_SRC - len(src))
    tgt_in = tgt_in[:MAX_LEN_TGT] + [PAD_IDX] * (MAX_LEN_TGT - len(tgt_in))
    tgt_out = tgt_out[:MAX_LEN_TGT] + [PAD_IDX] * (MAX_LEN_TGT - len(tgt_out))

    return src, tgt_in, tgt_out


In [9]:
class TranslationDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        eng, spa = self.pairs[idx]
        src, tgt_in, tgt_out = prepare_pair(eng, spa)

        return (
            torch.tensor(src, dtype=torch.long),
            torch.tensor(tgt_in, dtype=torch.long),
            torch.tensor(tgt_out, dtype=torch.long),
        )

In [10]:
# зробимо train/val/test спліт
random.shuffle(pairs)

total = len(pairs)
train_size = int(0.95 * total)


train_pairs = pairs[:train_size]
val_pairs   = pairs[train_size : ]


print("Train:", len(train_pairs))
print("Val:", len(val_pairs))

print("Total:", len(train_pairs) + len(val_pairs))


Train: 135781
Val: 7147
Total: 142928


In [11]:
train_ds = TranslationDataset(train_pairs)
val_ds   = TranslationDataset(val_pairs)

BATCH_SIZE = 64

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)


In [12]:
src, tgt_in, tgt_out = next(iter(train_loader))
print("src:", src.shape)       # [B, MAX_LEN_SRC]
print("tgt_in:", tgt_in.shape) # [B, MAX_LEN_TGT]
print("tgt_out:", tgt_out.shape)


src: torch.Size([64, 20])
tgt_in: torch.Size([64, 20])
tgt_out: torch.Size([64, 20])


In [13]:
def make_padding_mask(x, pad_idx=PAD_IDX):
    # x: [B, T]
    # True там, де пади (так хоче nn.Transformer)
    return (x == pad_idx)

def generate_subsequent_mask(size, device):
    # True = заборонено
    mask = torch.triu(torch.ones(size, size, dtype=torch.bool, device=device), diagonal=1)
    return mask

class TokenPositionalEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, max_len=50, dropout=0.1):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model, padding_idx=PAD_IDX)
        self.pos_emb = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: [B, T]
        B, T = x.shape
        pos = torch.arange(T, device=x.device).unsqueeze(0).expand(B, T)
        x = self.token_emb(x) + self.pos_emb(pos)
        return self.dropout(x)

In [14]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 src_vocab_size,
                 tgt_vocab_size,
                 d_model=64,
                 n_heads=4,
                 num_layers=2,
                 d_ff=128,
                 max_len_src=20,
                 max_len_tgt=20,
                 dropout=0.1):
        super().__init__()

        self.src_emb = TokenPositionalEmbedding(src_vocab_size, d_model, max_len_src, dropout)
        self.tgt_emb = TokenPositionalEmbedding(tgt_vocab_size, d_model, max_len_tgt, dropout)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            batch_first=True,
        )
        dec_layer = nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            batch_first=True,
        )

        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.decoder = nn.TransformerDecoder(dec_layer, num_layers=num_layers)

        self.output_proj = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt_in):
        """
        src: [B, S]
        tgt_in: [B, T]
        """
        src_key_padding_mask = make_padding_mask(src)      # [B, S]
        tgt_key_padding_mask = make_padding_mask(tgt_in)   # [B, T]

        # embeddings
        enc_in = self.src_emb(src)     # [B, S, D]
        dec_in = self.tgt_emb(tgt_in)  # [B, T, D]

        # encoder
        memory = self.encoder(
            enc_in,
            src_key_padding_mask=src_key_padding_mask
        )  # [B, S, D]

        # декодерська causal mask
        T = tgt_in.size(1)
        tgt_mask = generate_subsequent_mask(T, device=tgt_in.device)  # [T,T]

        dec_out = self.decoder(
            dec_in,
            memory,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask,
        )  # [B, T, D]

        logits = self.output_proj(dec_out)  # [B, T, V_tgt]
        return logits


In [15]:
src_vocab_size = len(eng_stoi)
tgt_vocab_size = len(spa_stoi)

model = Seq2SeqTransformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=64,
    n_heads=4,
    num_layers=4,
    d_ff=128,
    max_len_src=MAX_LEN_SRC,
    max_len_tgt=MAX_LEN_TGT,
).to(device)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Trainable parameters:", count_parameters(model))


Trainable parameters: 3232408


In [17]:
# for name, param in model.named_parameters():
#     print(f"{name:40s}  {param.numel()}")


In [18]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [19]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    n_tokens = 0

    for src, tgt_in, tgt_out in loader:
        src = src.to(device)
        tgt_in = tgt_in.to(device)
        tgt_out = tgt_out.to(device)

        optimizer.zero_grad()

        logits = model(src, tgt_in)  # [B, T, V]
        B, T, V = logits.shape

        loss = criterion(
            logits.view(B*T, V),
            tgt_out.view(B*T)
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * (tgt_out != PAD_IDX).sum().item()
        n_tokens += (tgt_out != PAD_IDX).sum().item()

    return total_loss / n_tokens


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    n_tokens = 0

    for src, tgt_in, tgt_out in loader:
        src = src.to(device)
        tgt_in = tgt_in.to(device)
        tgt_out = tgt_out.to(device)

        logits = model(src, tgt_in)
        B, T, V = logits.shape

        loss = criterion(
            logits.view(B*T, V),
            tgt_out.view(B*T)
        )

        total_loss += loss.item() * (tgt_out != PAD_IDX).sum().item()
        n_tokens += (tgt_out != PAD_IDX).sum().item()

    return total_loss / n_tokens


In [20]:
EPOCHS = 10

for ep in range(1, EPOCHS+1):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {ep}/{EPOCHS} - train loss/token: {train_loss:.4f}  val loss/token: {val_loss:.4f}")


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:515: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


Epoch 1/10 - train loss/token: 4.4845  val loss/token: 3.2712
Epoch 2/10 - train loss/token: 2.9972  val loss/token: 2.4903
Epoch 3/10 - train loss/token: 2.4028  val loss/token: 2.1366
Epoch 4/10 - train loss/token: 2.0817  val loss/token: 1.9449
Epoch 5/10 - train loss/token: 1.8806  val loss/token: 1.8200
Epoch 6/10 - train loss/token: 1.7396  val loss/token: 1.7385
Epoch 7/10 - train loss/token: 1.6341  val loss/token: 1.6877
Epoch 8/10 - train loss/token: 1.5490  val loss/token: 1.6349
Epoch 9/10 - train loss/token: 1.4760  val loss/token: 1.5971
Epoch 10/10 - train loss/token: 1.4184  val loss/token: 1.5630


In [21]:
def encode_src_sentence(text, stoi, max_len):
    tokens = tokenize(text)
    ids = [eng_stoi.get(t, UNK_IDX) for t in tokens]
    ids = ids[:max_len] + [PAD_IDX] * (max_len - len(ids))
    return torch.tensor(ids, dtype=torch.long).unsqueeze(0)  # [1, S]


def decode_tgt_ids(ids, itos):
    # ids: список індексів (наприклад, з BOS/EOS/PAD)
    tokens = []
    for i in ids:
        if i in (PAD_IDX, BOS_IDX, EOS_IDX):
            continue
        tokens.append(itos[i])
    return " ".join(tokens)


In [22]:
@torch.no_grad()
def translate(model, sentence, max_len=MAX_LEN_TGT):
    model.eval()
    device = next(model.parameters()).device

    # 1) кодуємо англійське речення
    src = encode_src_sentence(sentence, eng_stoi, MAX_LEN_SRC).to(device)  # [1, S]
    src_key_padding_mask = make_padding_mask(src)  # [1, S]

    # 2) пропускаємо через encoder
    enc_in = model.src_emb(src)  # [1, S, D]
    memory = model.encoder(enc_in, src_key_padding_mask=src_key_padding_mask)  # [1, S, D]

    # 3) старт декодера з BOS
    tgt_ids = [BOS_IDX]
    for _ in range(max_len):
        tgt_tensor = torch.tensor(tgt_ids, dtype=torch.long, device=device).unsqueeze(0)  # [1, T]
        tgt_key_padding_mask = make_padding_mask(tgt_tensor)

        T = tgt_tensor.size(1)
        tgt_mask = generate_subsequent_mask(T, device=device)  # [T, T]

        dec_in = model.tgt_emb(tgt_tensor)  # [1, T, D]
        dec_out = model.decoder(
            dec_in,
            memory,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask,
        )  # [1, T, D]

        logits = model.output_proj(dec_out[:, -1, :])  # [1, V]
        next_id = logits.argmax(dim=-1).item()

        tgt_ids.append(next_id)

        if next_id == EOS_IDX:
            break

    # 4) декодуємо у текст
    translation = decode_tgt_ids(tgt_ids, spa_itos)
    return translation


In [23]:
examples = [
    "Go.",
    "Hi!",
    "Good morning.",
    "Good night.",
    "I am fine.",
    "See you later.",
    "I forgot my keys at home.",
    "We are looking for a new apartment.",
    "She didn’t come because she was sick.",
]

for s in examples:
    print("EN:", s)
    print("ES:", translate(model, s))
    print()


EN: Go.
ES: vete.

EN: Hi!
ES: ¡hola!

EN: Good morning.
ES: buenas mañanas.

EN: Good night.
ES: buenas noches la noche.

EN: I am fine.
ES: estoy bien.

EN: See you later.
ES: ¡hasta luego.

EN: I forgot my keys at home.
ES: olvidé mis llaves en casa.

EN: We are looking for a new apartment.
ES: estamos buscando un nuevo apartamento.

EN: She didn’t come because she was sick.
ES: ella se adaptó porque estaba enfermo.

